In [1]:
import os
import sys
import getpass

user = getpass.getuser()
sys.dont_write_bytecode = True

sys.path.insert(0, '/Users/{}/Box/Utility Code'.format(user))

import utils_io as utils
import fiona
import arcgis as gis
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
work_dir = os.path.join('/Users',
                        user,
                       'Box',
                       'DataViz Projects',
                       'Spatial Analysis and Mapping',
                       'TOD TOC Policy Update Mapping')

In [3]:
toc_fgdb = os.path.join(work_dir, 
                        'TOD Policy Analysis', 
                        'TOD Policy Analysis.gdb')

## Pre-processing

- Pull General Plan and Zoning Capacity Data (Redshift)
- Pull Parcel Geometries (Socrata)
- Pull TOC Areas (BAM Portal)

In [4]:
pa_parcel_id = '5y7p-4hs4'

In [5]:
# create gdf from toc pda and tra feature class
pda_tra_gdf = gpd.read_file(filename=toc_fgdb, 
                            driver='File GDB', 
                            layer='TOC_PDA_TRA_epsg4326').to_crs('EPSG:26910')

In [6]:
sql = """
SELECT
  parcel_id,
  pa_apn_base as apn,
  county_id as pa_county,
  jurisdiction_id as pa_jurisdiction,
  zn_county,
  zn_jurisdiction,
  zn_recid,
  zn_code,
  zn_description,
  zn_regional_code,
  zn_regional_description,
  zn_min_dua,
  zn_max_dua,
  zn_min_far,
  zn_max_far,
  gp_recid,
  gp_code,
  gp_description,
  gp_regional_code,
  gp_regional_description,
  gp_min_dua,
  gp_max_dua,
  gp_min_far,
  gp_max_far
FROM
  basis_staging.parcel_base_tbl
  LEFT JOIN basis_staging.zn_base_tbl USING (zn_recid)
  LEFT JOIN basis_staging.zn_base_lot_properties USING (zn_recid)
  LEFT JOIN basis_staging.gp_base_tbl Using(gp_recid)
  LEFT JOIN basis_staging.gp_base_density Using (gp_recid)
"""
    
land_use_df = utils.pull_df_from_redshift_sql(sql_statement=sql)

took 1.8959 minutes


In [7]:
# pull parcel atlas parcel data as geojson
pa_parcels_geojson = utils.pull_geojson_from_socrata(socrata_data_id=pa_parcel_id)

pulling 2055059 rows as geojson
took 7.9544 minutes


In [ ]:
# convert parcel atlas parcel data to geodataframe
pa_parcels_gdf = gpd.GeoDataFrame.from_features(pa_parcels_geojson, 
                                               crs='EPSG:4326').to_crs('EPSG:26910')

In [ ]:
# # join parcels to land use data
# land_use_gdf = pd.merge(pa_parcels_gdf, 
#                         land_use_df, 
#                         how='left', 
#                         on='parcel_id')

## Perform largest area overlay